In [1]:
import cv2
import numpy as np

In [2]:
# harris角点检测不具有尺度不变性,SIFT具有尺度不变性,检测出的结果叫局部特征描述子,用一个数组来表述特征

![img](https://pic3.zhimg.com/v2-2394ea944751ce0d2be178e15798e9ae_r.jpg)


缺点：实时性不高、有时特征点较少、对边缘光滑的目标无法准确提取特征点。对模糊的图像和边缘平滑的图像，检测出的特征点过少，对圆更是无能为力。改进后的著名算法有SURF和CSIFT。

In [3]:
# 原理
# https://zhuanlan.zhihu.com/p/421792422
# https://www.cnblogs.com/wangguchangqing/p/4853263.html

In [ ]:
# 对原图进行多次下采样实现图像金字塔
# 对同一尺寸的图片施加不同σ的高斯滤波,实现不同清晰程度的图片采样
# 生成高斯差分金字塔(同一阶金字塔内部多张不同σ的高斯模糊图像,相邻两张之间做减法,求差分) DOG差分金字塔
    # 求差分为什么能体现关键点信息,对于不同的像素位置,同一位置两张相减,
    # 差值大的地方意味着该位置,图像区别比较大,能体现这这两张图不同之处的特征
# DOG 空间极值检测 
    # 针对每个点,检测起是否是本张及上下两张不同差分图3*3邻域内(8+9+9 =26)的极值点 


![image.png](https://pic2.zhimg.com/80/v2-4b7203cfc137b3c5d62257704b2cd271_720w.webp)
![image-2.png](https://pic2.zhimg.com/80/v2-4700bdc2d19353dded6e02e5df7b16b1_720w.webp)
![image-3.png](https://pic3.zhimg.com/80/v2-5cc18d4d6532b6deb8e2e1ef9cca667e_720w.webp)

In [ ]:
# 经过DOG极值点检测的得到的点是 一些离散的坐标点,相邻两个候选极值点之间(可能)隔了几个像素,
                                                    #(其所在位置的差分值不是不是真正的极值)
# 所以要通过对离散点进行连续函数的建模得到真正的极值点
# 利用二阶泰勒级数 f(x) = f(0)+f'(0)x+ f''(0)x^2/2

![img](https://pic2.zhimg.com/80/v2-8372f329cbd78d1787c3afd341664111_720w.webp)

In [4]:
# 去除弱响应的特征点
# 消除在落在差分图边缘上的极值点(和角点检测一样,用二次型判定是否在差分图图形边缘)
# 剩下的特征点就是保留的特征点

In [ ]:
# 对于得到的特征点 此时拥有(xyσ)三个信息
# 然后求这个点所在位置邻域(3*1.5σ)范围所有坐标的梯度的大小和方向(在差分图上计算),归类到8个方向上
    # 实际梯度直方图将0~360度的方向范围分为36个柱(bins)，其中每柱10度。
    
# 拥有最多统计数量的方向及拥有最多数量的方向80%以上的第二方向都将作为主方向(复制一份坐标和尺度)
#(拥有辅方向的关键点约占全部特征点的15%,但可以明显的提高关键点匹配的稳定性)

![img](https://pic1.zhimg.com/80/v2-57088e998772c1a08d746bfa57a8eb7c_720w.webp)

![img](https://pic4.zhimg.com/80/v2-c6f55218310e7a30c4075890d1759d63_720w.webp)

In [7]:
# 生成特征的描述
# 旋转坐标轴对齐某个特征点的主方向
# 特征描述子与特征点所在的尺度有关，因此，对梯度的求取应在特征点对应的高斯图像上进行
# 计算关键段周围(旋转后坐标下)16*16窗口内的梯度,16*16又分为16个4*4的种子窗口
# 每个种子窗口4*4->16个梯度信息->归类到8个方向->8维向量->16个8维向量->展平->128维向量


In [12]:
# 使用

# 创建sift对象
sift = cv2.xfeatures2d.SIFT_create() # <3.4.1.15 opencv-contrib-python
img = cv2.imread('../data/linna.jpg')

# 灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# 检测
kp = sift.detect(gray)
# print(kp) # 一个列表,里面是关键点对象
print(len(kp))

# 计算描述子
kp,des = sift.compute(gray,kp)
print(type(des),des.shape)

# 绘制关键点
cv2.drawKeypoints(img,kp,img)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 一步计算描述子和kp
kp,des = sift.detectAndCompute(gray,None)

1098
<class 'numpy.ndarray'> (1098, 128)


In [1]:
# 补充
# sift surf orb

In [3]:
surf = cv2.xfeatures2d.SURF_create()
img = cv2.imread('../data/linna.jpg')

# 灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
kp,des = surf.detectAndCompute(gray,None)

# 绘制关键点
cv2.drawKeypoints(img,kp,img)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
orb = cv2.ORB_create()
img = cv2.imread('../data/linna.jpg')

# 灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
kp,des = orb.detectAndCompute(gray,None)

# 绘制关键点
cv2.drawKeypoints(img,kp,img)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# 获取特征点和描述子后,可以对两幅图像进行特征匹配
# BF(Brute-Froce)暴力特征匹配方法
# img1中的每一个特征点都和img2中的每一个特征点之间计算描述子的向量距离(l1,l2,汉明距离....any_distance_function)
# 然后对img1中的每个描述子返回在img2中与其最小距离的那个

#特征匹配类
# BFMatcher(normType,crossCheck)
    # normType计算距离的方式    NORM_L1(sift surf),
        #                       NORM_L2(sift surf),
        #                       HAMMING(orb使用)  
    # crossCheck 是否进行交叉匹配,默认False
    
# 创建 特征匹配类 实例对象后
# 使用 match()方法进行匹配
# 该方法返回 DMatch对象
# DMatch对象.distance --描述符之间的距离
#  .trainIdx -目标图像描述符索引
#  .queryIdx -查询描述符索引
#  .imgIdx  -目标图像索引

# drawMatchs()绘制匹配特征点

In [39]:
img1 = cv2.imread('../data/linna.jpg')
img1 = cv2.resize(img1,(150,150))
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

img2 = cv2.imread('../data/linna.jpg')
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# 进行检测
sift = cv2.xfeatures2d.SIFT_create()
kp1,des1 = surf.detectAndCompute(gray1,None)
kp2,des2 = surf.detectAndCompute(gray2,None)

# 暴力匹配
bf = cv2.BFMatcher(cv2.NORM_L1,False)

# match(queryDescriptors, trainDescriptors[, mask]) -> matches
matches = bf.match(des1,des2)

print(type(matches))
print(len(matches))
print(len(des1))
print(len(des2))
# print(matches[0].distance,matches[0].imgIdx,matches[0].queryIdx,matches[0].trainIdx)
outImg =cv2.drawMatches(img1,kp1,img2,kp2,matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

<class 'list'>
181
181
1462


In [40]:
# bf对象的 knnmatch 方法
matches = bf.knnMatch(des1,des2,k = 2) # 每个描述子会返回k个匹配到的描述子
print(type(matches))
print(len(matches))
print(len(des1))
print(len(des2))
outImg =cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

<class 'list'>
181
181
1462


In [42]:
# FLANN特征匹配 快速最近邻

In [ ]:
# 略

In [43]:
# 筛选部分配对的特征进行绘图

# 找一个特征点少点的图来试验


![img](../data/R-C.jpg)
![img](../data/R-C-2.png)

In [50]:
img1 = cv2.imread('../data/R-C.jpg')
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2 = cv2.imread('../data/R-C-2.png')
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SIFT_create()
kp1,des1 = sift.detectAndCompute(gray1,None)
kp2,des2 = sift.detectAndCompute(gray2,None)

out1 = cv2.drawKeypoints(img1,kp1,None)
out2 = cv2.drawKeypoints(img2,kp2,None)

# cv2.imshow('img',out2)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

bf = cv2.BFMatcher(cv2.NORM_L1,False)
matches = bf.match(des2,des1)

outImg =cv2.drawMatches(img2,kp2,img1,kp1,matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image.png](001.png)

In [58]:
# 直接所有的配对项进行匹配错误的匹配较多
# 选择比平均距离小的
distance_list = [m.distance for m in matches]
mean_distance = sum(distance_list)/len(distance_list)
print(mean_distance)
good_matches = [m for m in matches if m.distance<mean_distance]

outImg =cv2.drawMatches(img2,kp2,img1,kp1,good_matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

711.6835443037975


少了一些错误,但是还不够完善
![image-2.png](002.png)

In [74]:
# 选择匹配距离前30%的
distance_list = [m.distance for m in matches]
idx = np.argsort(np.array(distance_list))
good_idx = idx[0:int(0.3*len(distance_list))]
# print(good_idx)
good_matches = []
for i in good_idx:
    good_matches.append(matches[i])

outImg =cv2.drawMatches(img2,kp2,img1,kp1,good_matches,None)
cv2.imshow('img',outImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image.png](003.png)